In [5]:
import numpy
import cv2
from gco import pygco
import matplotlib.pyplot as plt
import os
import typing
import math
from numpy.typing import NDArray

# @cache
def compute_distance(point1: numpy.ndarray, point2: numpy.ndarray) -> float:
    """
    compute distance of two pixels
    Parameters
    ----------
    point1: ndarray, uint8, shape=(3)
        point one
    point2: ndarray, uint8, shape=(3)
        point two
    """
    # return (abs(point1[0]-point2[0]) + abs(point1[1] - point2[1]) + abs(point1[2] - point2[2]) ) / 3 / 255
    # if not dividing 255 as the C pseudocode example, there will be a overflowing of MAX_VALUE of Integer when calling pycgo.cut_grid_graph
    return numpy.sum(numpy.abs(point1-point2)) /3 / 255

In [6]:
ROAD_FILE_PATH = "Road"
ROAD_FRAME_PATH = f"{ROAD_FILE_PATH}{os.sep}src"
CAMERAS_PARAMETERS_PATH = f"{ROAD_FILE_PATH}{os.sep}cameras.txt"
file_list: typing.List[str] = os.listdir(path=ROAD_FRAME_PATH)
file_list.sort()
print(f"file_list:{file_list}")
frame_num: int = len(file_list)
camera_paramters:numpy.mat = \
    numpy.mat(data=numpy.loadtxt(fname=CAMERAS_PARAMETERS_PATH, 
                                 dtype=numpy.float64, skiprows=1),
              dtype=numpy.float64)
K = []
R = []
T = []
for i in range(frame_num):
    K.append(camera_paramters[7*i:7*i+3, :])
    R.append(camera_paramters[7*i+3:7*i+6, :])
    T.append(camera_paramters[7*i+6, :])
print(f"k:{len(K)} file_list:{len(file_list)}")
camera_paramters = None
print(f"road {ROAD_FRAME_PATH}{os.sep}{file_list[0]}")
img0: numpy.ndarray = cv2.imread(f"{ROAD_FRAME_PATH}{os.sep}{file_list[0]}", cv2.IMREAD_COLOR)
img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
# img2 = cv2.resize(img2, (width1, height1))
BASE_HEIGHT, BASE_WIDTH, _ = img0.shape


file_list:['test0000.jpg', 'test0001.jpg', 'test0002.jpg', 'test0003.jpg', 'test0004.jpg', 'test0005.jpg', 'test0006.jpg', 'test0007.jpg', 'test0008.jpg', 'test0009.jpg', 'test0010.jpg', 'test0011.jpg', 'test0012.jpg', 'test0013.jpg', 'test0014.jpg', 'test0015.jpg', 'test0016.jpg', 'test0017.jpg', 'test0018.jpg', 'test0019.jpg', 'test0020.jpg', 'test0021.jpg', 'test0022.jpg', 'test0023.jpg', 'test0024.jpg', 'test0025.jpg', 'test0026.jpg', 'test0027.jpg', 'test0028.jpg', 'test0029.jpg', 'test0030.jpg', 'test0031.jpg', 'test0032.jpg', 'test0033.jpg', 'test0034.jpg', 'test0035.jpg', 'test0036.jpg', 'test0037.jpg', 'test0038.jpg', 'test0039.jpg', 'test0040.jpg', 'test0041.jpg', 'test0042.jpg', 'test0043.jpg', 'test0044.jpg', 'test0045.jpg', 'test0046.jpg', 'test0047.jpg', 'test0048.jpg', 'test0049.jpg', 'test0050.jpg', 'test0051.jpg', 'test0052.jpg', 'test0053.jpg', 'test0054.jpg', 'test0055.jpg', 'test0056.jpg', 'test0057.jpg', 'test0058.jpg', 'test0059.jpg', 'test0060.jpg', 'test0061.jpg

## process 5 frames

In [7]:
NUM_FRAMES_PROCESSED: int = 5
# NUM_FRAMES_PROCESSED: int = 5
# 0-1
SIGMA_C = 1
SIGMA_D = 2.5   
SIGMAD_D_2 = 2 * (SIGMA_D**2)
EPSILON = 50

STEP = 50
D_MAX = 0.02
DISPARITY_RANGE: float = D_MAX / STEP

WS = 1
EPSILON_LAMBDA = 0.001
EPSILON_MU = 0.001
MAX_DIFF = 1
LAMBDA = 0.1
# @functools.cache
def compute_pc(difference: float) -> float:
    """
        Args:
            disparity: |It(x) - It'(ltt'(x,d))| >=0
    """
    return SIGMA_C / (SIGMA_C + difference)

# @functools.cache
def compute_pv(distance:int)-> float:
    return math.exp((-(distance))/SIGMAD_D_2)

# @functools.cache
def get_paramters(origin_idx: int, target_idx: int)-> typing.Tuple[numpy.mat, numpy.mat]:
    tem_k1 = K[origin_idx]
    tem_r1 = R[origin_idx]
    tem_t1 = T[origin_idx]
    tem_k2 = K[target_idx]
    tem_r2 = R[target_idx]
    tem_t2 = T[target_idx]
    tem_point_part_matrice = tem_k2 @ tem_r2.T @ tem_r1 @ numpy.linalg.inv(tem_k1)
    tem_epipolar_line_matrice = tem_k2 @ tem_r2.T @ numpy.mat(tem_t1-tem_t2).T
    return tem_point_part_matrice, tem_epipolar_line_matrice

# @functools.cache
def get_epipolar_line(origin_idx: int, target_idx: int, d: float)->numpy.ndarray:
    _, tem_epipolar_line_matrice = get_paramters(origin_idx=origin_idx, target_idx=target_idx)
    epipolar_line = d * tem_epipolar_line_matrice
    return epipolar_line

# @functools.cache
def compute_mu(difference: float) -> float:

    pass

def _compute_distance(img1:numpy.ndarray, img2: numpy.ndarray) -> numpy.ndarray:
    # return numpy.sum(numpy.abs(img_a - img_b), axis=-1, keepdims=True)
    return numpy.sqrt(numpy.sum(numpy.square(img1-img2), axis=-1))


def _generate_fp(step:int,threshold:int):
    fp = numpy.zeros(shape=(step,step), dtype=numpy.int32)
    for i in range(step):
        for j in range(step):
            if i!= j:
                fp[i][j] = min(abs(i-j),threshold)
    return fp

def _get_img_color(img: numpy.ndarray, position: numpy.ndarray):
    # position = numpy.transpose(position[:2,:,:], [1,2,0])
    return cv2.remap(src=img, map1=numpy.float32(position), map2=None, interpolation=cv2.INTER_NEAREST, borderValue=[128, 128, 128])

def compute(start_frame_num: int, is_output_result: bool = True) -> typing.Tuple[typing.List[numpy.ndarray], typing.List[numpy.ndarray], typing.List[numpy.ndarray]]:
    pc_list: typing.List[numpy.ndarray] = []
    labels_list: typing.List[numpy.ndarray] = []
    start = start_frame_num
    end = start + (NUM_FRAMES_PROCESSED - 1)
    # tem_k: typing.List[numpy.mat] = K[start:end+1]
    # tem_r: typing.List[numpy.mat] = R[start:end+1]
    # tem_t: typing.List[numpy.mat] = T[start:end+1]
    K1 = K[start]
    R1 = R[start]
    T1 = T[start].T
    K1_inv: numpy.mat =  numpy.mat(data=numpy.linalg.inv(K1))
    img_list= [None] * NUM_FRAMES_PROCESSED
    L = None
    D= None
    for i in range(NUM_FRAMES_PROCESSED):
        idx = i + start
        img: numpy.ndarray = cv2.imread(f"{ROAD_FRAME_PATH}{os.sep}{file_list[idx]}", cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # img = cv2.resize(src=img, dsize=(BASE_WIDTH, BASE_HEIGHT))
        img_list[i] = img
    img0: numpy.ndarray = img_list[0]
    height0, width0, _ = img0.shape
    L = numpy.zeros([STEP, height0, width0], dtype=numpy.float64)
    num = 0
    num_images_per_row = 5
    plt.figure(figsize=(100, 80))  # 调整图像大小
    Hr = numpy.sum((img0[:, 0:-1].astype(numpy.float64) - img0[:, 1:].astype(numpy.float64))**2, axis=2)
    Hr = 1.0/numpy.sqrt(Hr+EPSILON)
    Ver = numpy.sum((img0[0:-1, :].astype(numpy.float64) -img0[1:, :].astype(numpy.float64))**2, axis=2)
    Ver = 1.0/numpy.sqrt(Ver+EPSILON)
    
    for i in range(1, NUM_FRAMES_PROCESSED):
        idx = start + i
        K2 = K[idx]
        R2 = R[idx]
        T2 = T[idx].T
        X,Y = numpy.int32(numpy.meshgrid(numpy.arange(width0, dtype=numpy.int32),
                                         numpy.arange(height0, dtype=numpy.int32)))
        X, Y, Z = (numpy.expand_dims(a=X, axis=0), numpy.expand_dims(a=Y, axis=0), numpy.ones(shape=(1, height0, width0), dtype=numpy.int32))
        homo_coordinates: numpy.ndarray = numpy.concatenate((X, Y, Z), axis=0)
        # pc_tem_array = numpy.zeros()
        for s in range(STEP):
            d = s * DISPARITY_RANGE
            d_arr = numpy.full(shape=(1, height0 *width0), fill_value=d, dtype=numpy.float64)
            coorsxy = numpy.reshape(homo_coordinates, [3,-1])
            depth = (T1-T2) @ d_arr
            projected_pos = (K1*R2.T) * ((R1*K1_inv) * coorsxy + depth)
            projected_pos = numpy.divide(projected_pos, projected_pos[2, :])
            projected_pos = numpy.reshape(numpy.asarray(projected_pos), [3, height0, width0])
            projected_pos = numpy.transpose(projected_pos[:2,:,:], [1,2,0])

            color_img2 = _get_img_color(img=img_list[i] , position=projected_pos)
            color_distance = _compute_distance(color_img2, img0)
            pc = SIGMA_C/(SIGMA_C + color_distance)
            pc_list.append(pc)
            L[s,:,:] += pc[:,:]
        u = numpy.reciprocal(L.max(axis=0, keepdims=True))    
        unary= 1 - u * L
        # unary = 1 - (unary / numpy.max(unary))
        confine_unary=numpy.zeros((height0, width0, STEP))
        for s in range(STEP):
            confine_unary[:,:,s] = unary[s,:,:]
        fp = _generate_fp(step=STEP, threshold=10)
        # print(f"basic confine_unary:{confine_unary}")
        labels = pygco.cut_grid_graph_simple(confine_unary, fp*LAMBDA, connect=8, n_iter=-1)
        labels = labels.reshape(height0, width0)
        labels_list.append(labels)
        # D[i,:,:] = labels
        if is_output_result:
            if num < num_images_per_row:
                plt.subplot(1, num_images_per_row, num + 1)
                plt.imshow(labels, cmap='gray')
                plt.title(f'Image {idx}')
                plt.axis('off')
                num += 1
            else:
                # 显示当前行，并开始新的一行
                plt.show()
                plt.figure(figsize=(100, 80))  # 调整图像大小
                plt.subplot(1, num_images_per_row, 1)
                plt.imshow(labels, cmap='gray')
                plt.title(f'Image {idx}')
                plt.axis('off')
                num = 1  # 重置计数器并包含当前图像
    if is_output_result and num > 0:
        plt.show()
    return pc_list, labels_list, img_list

# init_depth_map_list: typing.List[numpy.ndarray] = compute(start_frame_num=0)


In [8]:
LAMBDA_BUNDLE = 0.00001
def compute_with_bundle_optimize(start_frame_num:int) -> None:
    # index 0 -> there is no depth map
    # pc_list,labels_list, img_list = compute(start_frame_num=0, is_output_result=False)
    pc_list,labels_list, img_list = compute(start_frame_num=start_frame_num, is_output_result=True)
    pc_list.insert(0, None)
    labels_list.insert(0, None)
    start = start_frame_num
    end = start + NUM_FRAMES_PROCESSED
    K1 = K[start]
    R1 = R[start]
    T1 = T[start].T
    K1_inv: numpy.mat =  numpy.mat(data=numpy.linalg.inv(K1))
    img0: numpy.ndarray = img_list[0]
    height0, width0, _ = img0.shape
    Hr = numpy.sum((img0[:, 0:-1].astype(numpy.float64) - img0[:, 1:].astype(numpy.float64))**2, axis=2)
    Hr = 1.0/numpy.sqrt(Hr+EPSILON)
    Ver = numpy.sum((img0[0:-1, :].astype(numpy.float64) -img0[1:, :].astype(numpy.float64))**2, axis=2)
    Ver = 1.0/numpy.sqrt(Ver+EPSILON)
    X,Y = numpy.int32(numpy.meshgrid(numpy.arange(width0, dtype=numpy.int32),
                                     numpy.arange(height0, dtype=numpy.int32)))
    X, Y, Z = (numpy.expand_dims(a=X, axis=0), numpy.expand_dims(a=Y, axis=0), numpy.ones(shape=(1, height0, width0), dtype=numpy.int32))
    plt.figure(figsize=(100, 80))  # 调整图像大小
    num = 0
    num_images_per_row = 5
    num_rows = (NUM_FRAMES_PROCESSED + num_images_per_row - 1) // num_images_per_row
    for i in range(1, NUM_FRAMES_PROCESSED):
        L = numpy.zeros((STEP, height0, width0), dtype=numpy.float64) #[d_numbers, h, w]
        num +=1
        idx = start + i
        K2 = K[idx]
        R2 = R[idx]
        T2 = T[idx].T
        K2_inv: numpy.mat =  numpy.mat(data=numpy.linalg.inv(K2))
        plt.figure(figsize=(100, 80))  # 调整图像大小

        for s in range(STEP):
            d = s * DISPARITY_RANGE
            pc = pc_list[i]
            d_arr = numpy.full(shape=(1, height0 *width0), fill_value=d, dtype=numpy.float64)
            homo_coordinates: numpy.ndarray = numpy.concatenate((X, Y, Z), axis=0)
            coorsxy = numpy.reshape(homo_coordinates, [3,-1])
            print(f"coorsxy:{coorsxy.shape}")
            loc_2 = numpy.dot(numpy.dot(numpy.dot(numpy.dot(K2, R2.T), R1), numpy.linalg.inv(K1)), coorsxy) + numpy.dot(numpy.dot(K2, R2.T), (T1 - T2))* d
            loc_2 = numpy.divide(loc_2, loc_2[2:])
            label1 = labels_list[i]
            loc_2_reshape =  numpy.reshape(numpy.asarray(loc_2), [3, height0, width0])
            loc_2_reshape[0, loc_2_reshape[0, :] < 0] = 0
            loc_2_reshape[1, loc_2_reshape[1, :] < 0] = 0
            loc_2_reshape[0, loc_2_reshape[0, :] >= width0] = width0 - 1
            loc_2_reshape[1, loc_2_reshape[1, :] >= height0] = height0 - 1
            print(f"loc_2_reshape:{loc_2_reshape.shape}")
            loc_2_reshape_get_match = numpy.transpose(loc_2_reshape[:2,:,:], [1,2,0])
            print(f"loc_2_reshape_get_match:{loc_2_reshape_get_match}")
            print(f"label1:{label1.shape}")
            print(f"loc_2_reshape_get_match:{loc_2_reshape_get_match.shape}")
            label_projected = _get_img_color(img=label1 , position=loc_2_reshape_get_match)
            print(f"label_projected:{label_projected.shape}")
            disparity_projected = label_projected * DISPARITY_RANGE
            disparity_projected = disparity_projected.reshape(1, -1)
            print(f"coorsxy22:{coorsxy}")
            loc_back = numpy.dot(numpy.dot(numpy.dot(numpy.dot(K1, R1.T), R2), numpy.linalg.inv(K2)), coorsxy) + numpy.dot(numpy.dot(numpy.dot(K1, R1.T), (T2 - T1)), disparity_projected)
            loc_back = numpy.divide(loc_back, loc_back[2:])
            loc_back =  numpy.reshape(numpy.asarray(loc_back), [3, height0, width0])
            loc_back[0, loc_back[0, :] < 0] = 0
            loc_back[1, loc_back[1, :] < 0] = 0
            loc_back[0, loc_back[0, :] >= width0] = width0 - 1
            loc_back[1, loc_back[1, :] >= height0] = height0 - 1
            # depth = (T1-T2) @ d_arr
            # remap = (K1*R2.T) * ((R1*K1_inv) * coorsxy + depth)
            # remap = numpy.divide(remap, remap[2, :])
            # remap = numpy.reshape(numpy.asarray(remap), [3, height0, width0])
            # coorsxy_inv = numpy.reshape(coorsxy_inv, [3,-1])
            # depth = (T2-T1) @ d_arr
            # tem = (K2*R1.T) * ((R2*K2_inv) * coorsxy + depth)
            # tem = numpy.divide(tem, tem[2, :])
            # tem = numpy.reshape(numpy.asarray(tem), [3, height0, width0])
            loc_back = loc_back.transpose(1, 2, 0)
            homo_coordinates = homo_coordinates.transpose(1, 2, 0)
            print(f"homo_coordinates:{homo_coordinates.shape}")
            print(f"loc_back:{loc_back.shape}")
            coor_dist = numpy.sum(numpy.square(homo_coordinates-loc_back), axis=-1)
            # coor_dist = _compute_distance(img1=homo_coordinates,img2=loc_back)
            pv = numpy.exp(coor_dist/(-SIGMAD_D_2))
            print(f"pc:{pc.shape}, pv:{pv.shape}")
            L[s,:,:] += (pc*pv)[:,:]
    
    u = numpy.reciprocal(L.max(axis=0, keepdims=True))
    unary= 1 - u*L
    confine_unary=numpy.zeros((height0, width0, STEP))
    cur_count_confie = 1
    print(f"confine_unary:{confine_unary.shape}")
    print(f"height0:{height0} width0:{width0}")
    for s in range(STEP):
        confine_unary[:,:,s] = unary[s,:,:]
    # confine_unary = numpy.log(confine_unary)
    for level1 in range(confine_unary.shape[0]):
        for level2 in range(confine_unary.shape[1]):
            cur_count_confie += 1
            if cur_count_confie > 30:
                break
            print(confine_unary[level1, level2])
    fp = _generate_fp(step=STEP, threshold=10)
    # min_val = numpy.min(confine_unary)
    # max_val = numpy.max(confine_unary)
    # confine_unary = ((confine_unary - min_val) / (max_val - min_val)) 
    labels = pygco.cut_grid_graph(unary_cost=confine_unary, pairwise_cost=fp*LAMBDA_BUNDLE, cost_v=Ver, cost_h=Hr, n_iter=-1, algorithm="expansion")
    # labels = pygco.cut_grid_graph_simple(confine_unary, fp*LAMBDA_BUNDLE, connect=8, n_iter=-1)
    labels = labels.reshape(height0, width0)
    plt.imshow(labels, cmap='gray')
    plt.title(f'Image Bundle from {start} to {end}')
    plt.show()
    # if num < num_images_per_row:
    #     plt.subplot(1, num_images_per_row, num + 1)
    #     plt.imshow(labels, cmap='gray')
    #     plt.title(f'Image {idx}')
    #     plt.axis('off')
    #     num += 1
    # else:
    #     plt.show()
    #     plt.figure(figsize=(100, 80)) 
    #     plt.subplot(1, num_images_per_row, 1)
    #     plt.imshow(labels, cmap='gray')
    #     plt.title(f'Image {idx}')
    #     plt.axis('off')
    #     num = 1  # 
    # # plt.subplot(1, num_images_per_row, num%num_images_per_row + 1)
    # # plt.imshow(labels,cmap='gray')
    # # plt.title(f'frame:{file_list[idx]} Disparity Initialization with Bundle Optimization')
    # # plt.show()
    # if num > 0:
    #     plt.show()
compute_with_bundle_optimize(start_frame_num=0)
# for sig in [2, 200,400,800,1000, 10000,]:
#     global SIGMA_D, SIGMAD_D_2
#     SIGMA_D = sig
#     SIGMAD_D_2 = 2 * (SIGMA_D**2)
#     for l in [1, 0.1, 0.01,0.000000001,0.0000000001]:
#         if sig == 2 and l >= 0.02:
#             continue
#         LAMBDA_BUNDLE = l
#         compute_with_bundle_optimize(start_frame_num=0)